In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
sys.path.insert(0, os.path.dirname(os.path.abspath("")))

In [3]:
import rlcard
from rlcard.agents import RandomAgent

In [4]:
import tute.rlcard
from tute.rlcard.agent import TuteHumanAgent

In [5]:
import logging
tute.logger.setLevel(logging.INFO)

In [6]:
env = rlcard.make('tute')

In [7]:
print("Number of actions:", env.num_actions)
print("Number of players:", env.num_players)
print("Shape of state:", env.state_shape)
print("Shape of action:", env.action_shape)

Number of actions: 40
Number of players: 2
Shape of state: [[41, 9], [41, 9]]
Shape of action: [None, None]


In [8]:
random_agent = RandomAgent(num_actions=env.num_actions)
human_agent = TuteHumanAgent(player=1, tute=env.game)
env.set_agents([random_agent, human_agent])

In [9]:
trajectories, player_wins = env.run(is_training=False)


================= Trump =================
el cuatro de bastos


=============== Your Hand ===============
1: el as de bastos
2: el rey de bastos
3: el caballo de copas
4: el cuatro de copas
5: el tres de espadas
6: el dos de espadas
7: el dos de oros
8: la sota de bastos
1  2  3  4  5  6  7  8 ? 6

Player 2 played el dos de espadas
Player 1 played el rey de copas
Player 2 won trick


================= Trump =================
el cuatro de bastos


=============== Your Hand ===============
1: el as de bastos
2: el rey de bastos
3: el caballo de copas
4: el cuatro de copas
5: el tres de espadas
6: el dos de oros
7: la sota de bastos
8: el caballo de bastos
1  2  3  4  5  6  7  8 ? 6

Player 2 played el dos de oros
Player 1 played el seis de bastos
Player 2 canta bastos
Player 2 won trick


================= Trump =================
el cuatro de bastos


=============== Your Hand ===============
1: el as de bastos
2: el rey de bastos
3: el caballo de copas
4: el cuatro de copas
5: el tres 

In [11]:
player_wins

[0, 150]